In [1]:
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

In [2]:
# Load images
lena = cv2.imread("lena.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32)
baboon = cv2.imread("baboon.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32)

# Compute DCT
lena_dct = cv2.dct(lena)
baboon_dct = cv2.dct(baboon)


In [3]:
fc_list = [256, 128, 64, 32, 16, 8, 4, 2]

# NCC function
def ncc(img1, img2):
    img1_mean = np.mean(img1)
    img2_mean = np.mean(img2)
    return np.sum((img1 - img1_mean)*(img2 - img2_mean)) / np.sqrt(np.sum((img1 - img1_mean)**2) * np.sum((img2 - img2_mean)**2))

results = []

for fc in fc_list:
    mask_low = np.zeros_like(lena_dct)
    mask_low[:fc, :fc] = 1

    # Mixed images
    lena_baboon = cv2.idct(mask_low * lena_dct + (1 - mask_low) * baboon_dct)
    baboon_lena = cv2.idct(mask_low * baboon_dct + (1 - mask_low) * lena_dct)

    # Clip values to [0,255] and convert to uint8 for metrics
    lena_baboon_clipped = np.clip(lena_baboon, 0, 255)
    baboon_lena_clipped = np.clip(baboon_lena, 0, 255)

    # Compute metrics
    results.append([
        fc,
        psnr(lena, lena_baboon_clipped, data_range=255), 
        ssim(lena, lena_baboon_clipped, data_range=255), 
        ncc(lena, lena_baboon_clipped),
        psnr(baboon, baboon_lena_clipped, data_range=255), 
        ssim(baboon, baboon_lena_clipped, data_range=255), 
        ncc(baboon, baboon_lena_clipped)
    ])

# Display results table
df = pd.DataFrame(results, columns=[
    "fc", 
    "PSNR (Lena low/Baboon high)", "SSIM (Lena low/Baboon high)", "NCC (Lena low/Baboon high)",
    "PSNR (Baboon low/Lena high)", "SSIM (Baboon low/Lena high)", "NCC (Baboon low/Lena high)"
])

print(df)

    fc  PSNR (Lena low/Baboon high)  SSIM (Lena low/Baboon high)  \
0  256                    24.084586                     0.543393   
1  128                    20.791609                     0.381756   
2   64                    19.078657                     0.263172   
3   32                    17.608062                     0.193260   
4   16                    16.129283                     0.150391   
5    8                    14.356735                     0.132432   
6    4                    13.166247                     0.123649   
7    2                    12.360340                     0.120465   

   NCC (Lena low/Baboon high)  PSNR (Baboon low/Lena high)  \
0                    0.948109                    24.061084   
1                    0.894004                    20.730655   
2                    0.843255                    18.994632   
3                    0.775462                    17.530229   
4                    0.669413                    16.082525   
5              